In [95]:
!pip3 install pandas
!pip3 install SQLAlchemy
!pip3 install PyMySQL
!pip3 install pyodbc
!pip3 install seaborn
import seaborn as sns
import numpy as np
import pymysql
from urllib.parse import quote 
import mysql.connector
import sqlalchemy
from sqlalchemy.engine import create_engine
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 288 kB 554 kB/s eta 0:00:01
     |████████████████

In [87]:
#connect to mysql servel
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Parvashah@123",
  database="yelpdb"
)

print(mydb)

mycursor = mydb.cursor()


In [88]:
# Create a an engine for more optimized insertion of data through SQLalchemy
constring = 'mysql+mysqlconnector://root:%s@localhost:3306/yelpdb' % quote('Parvashah@123')
dbEngine = sqlalchemy.create_engine(constring, connect_args={'connect_timeout': 10}, echo=False)

In [89]:
# Testing the connection of the engine
try:
    with dbEngine.connect() as con:
        con.execute("SELECT 1")
    print('engine is valid')
except Exception as e:
    print(f'Engine invalid: {str(e)}')

engine is valid


In [42]:
# Table Creeation
# create business table
mycursor.execute("DROP TABLE IF EXISTS business")
mycursor.execute("CREATE TABLE business (business_id VARCHAR(255) PRIMARY KEY, name VARCHAR(255), address VARCHAR(255), city VARCHAR(255), state TEXT(2), postal_code INT(5), stars DECIMAL(10,1), review_count INT, is_open BOOL)")
# create category table
mycursor.execute("DROP TABLE IF EXISTS category")
mycursor.execute("CREATE TABLE category (business_id VARCHAR(255),categories VARCHAR(255), PRIMARY KEY (business_id, categories),CONSTRAINT cat_b_id_fk FOREIGN KEY (business_id) REFERENCES business (business_id) ON UPDATE CASCADE ON DELETE CASCADE)")
# create user table
mycursor.execute("DROP TABLE IF EXISTS user")
mycursor.execute("CREATE TABLE user (user_id VARCHAR(255) PRIMARY KEY, name VARCHAR(255), review_count INT, yelping_since DATETIME, average_stars FLOAT)")
# create review table
mycursor.execute("DROP TABLE IF EXISTS review")
mycursor.execute("CREATE TABLE review (review_id VARCHAR(255) PRIMARY KEY, user_id VARCHAR(255), business_id VARCHAR(255), stars INT(1), text TEXT(65535), time DATETIME, CONSTRAINT user_id_fk FOREIGN KEY (user_id) REFERENCES user (user_id) ON UPDATE CASCADE ON DELETE CASCADE, CONSTRAINT business_id_fk FOREIGN KEY (business_id) REFERENCES business (business_id) ON UPDATE CASCADE ON DELETE CASCADE)")

In [43]:
# Business dataframe creation
df_b = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
# delete unwanted columns
del df_b['attributes'], df_b['hours'], df_b['categories'], df_b['latitude'], df_b['longitude']

In [44]:
# category dataframe creation
df_p = pd.read_json('yelp_dataset/yelp_academic_dataset_business.json', lines=True)
cat_info = []
for index, row in df_p.iterrows():
    try:
        #split the category values by "," and append in list
        for cat in row["categories"].split(","):
            cat_info.append((row["business_id"],cat))
    except:
        continue
df_cat = pd.DataFrame(cat_info,columns=['business_id','categories'])


In [45]:
# Insert data into business table from json file
df_b.to_sql(con=dbEngine, schema="yelpdb", name="temporary_table", if_exists="replace", index=False, chunksize=1000)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO business (SELECT * FROM temporary_table)'
    cnx.execute(insert_sql)

In [46]:
# create user dataframe
chunks = pd.read_json('yelp_dataset/yelp_academic_dataset_user.json', lines=True, chunksize = 5000)
users = pd.DataFrame()
for chunk in chunks:
    users = pd.concat([users, chunk])
    del users['useful'], users['funny'], users['cool'], users['elite'], users['friends'], users['fans'], users['compliment_hot'], users['compliment_more'], users['compliment_profile'], users['compliment_cute'], users['compliment_list'], users['compliment_note'], users['compliment_plain'], users['compliment_cool'], users['compliment_funny'], users['compliment_writer'], users['compliment_photos']
    

In [47]:
# Insert data into user table from user dataframe
users.to_sql(con=dbEngine, schema="yelpdb", name="temporary_table", if_exists="replace", index=False, chunksize=5000)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO user (SELECT * FROM temporary_table)'
    cnx.execute(insert_sql)

In [48]:
# Review json has more than 6 M lines of data to handle that we need to take additional measures and create review dataframe
# dataframe creation is done in chunks since it is a large file which cannot be read at once. 
# In addition we specify the datatype so it takes considerably less time
b_pandas = []
r_dtypes = {"review_id": np.string_,
            "user_id": np.string_,
            "business_id": np.string_,
            "stars": np.int32, 
            "text": np.string_, 
            "time": np.datetime64
           }
with open("yelp_dataset/yelp_academic_dataset_review.json", "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          dtype=r_dtypes, chunksize=5000)
        
    for chunk in reader:
        reduced_chunk = chunk.drop(columns=['useful', 'funny','cool'])
        b_pandas.append(reduced_chunk)
    
df_review = pd.concat(b_pandas, ignore_index=True)

In [1]:
review_sample = df_review.sample(10000)

NameError: name 'df_review' is not defined

In [91]:
#Insert data into review table from the dataframe
review_sample.to_sql(con=dbEngine, schema="yelpdb", name="temporary_table", if_exists="replace", index=False, chunksize=10000)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO review (SELECT * FROM temporary_table)'
    cnx.execute(insert_sql)

In [92]:
#Insert data into category table from the dataframe
df_cat.to_sql(con=dbEngine, schema="yelpdb", name="temporary_table", if_exists="replace", index=False, chunksize=10000)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO category (SELECT * FROM temporary_table)'
    cnx.execute(insert_sql)